<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection**

## **The Issue**

In [ ]:
#Scrapping works well with stuctured data, like tables etc.
#Also, there is the issue of labelling:
#Using myself or someelse may result in bias (interpretation of swahili words or sentences based on given labels)
  #Also, this will be tedious and costly (time)

In [ ]:
#Any other alternative???

# **Data Collection Methods**

## **1.0 Using Scrapy**

### **1.0.0 Libraries**

In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.7.15'

### **1.0.1 More Libraries - Scrapy**

In [2]:
#Import Scrapy
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 6.8 MB/s 
     |████████████████████████████████| 3.1 MB 54.2 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 93 kB 2.6 MB/s 
     |████████████████████████████████| 4.0 MB 42.7 MB/s 
     |████████████████████████████████| 254 kB 56.1 MB/s 
     |████████████████████████████████| 74 kB 3.3 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=0df3e565744c326c8cf184f73eb9dcb30cbb558d211ef82f38931503ae76735f
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


### **1.0.2 Setting Up the Pipeline**

In [15]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### **1.0.3 Defining the Spider**

In [16]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://artsandculture.google.com/story/learn-swahili-21-inspirational-kanga-sayings/HgJClAZer2a1Lw',
        'https://www.tuko.co.ke/271956-very-funny-swahili-quotes-images.html',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

### **1.0.4 Starting the Crawler**

In [17]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 10:57:28 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-27 10:57:28 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-10-27 10:57:28 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGE

ReactorNotRestartable: ignored

### **1.0.5 Checking the Files**

In [18]:
ll quoteresult.*

-rw-r--r-- 1 root 0 Oct 27 10:57 quoteresult.jl
-rw-r--r-- 1 root 0 Oct 27 10:52 quoteresult.json


In [19]:
!tail -n 2 quoteresult.jl

In [20]:
!tail -n 2 quoteresult.json

### **1.0.6 Turning them to Dataframes**

In [21]:
import pandas as pd


In [22]:
dfjson = pd.read_json('quoteresult.json')


ValueError: ignored

## **2.0 Scrapy 2.0**

### **2.0.0 Libraries**

In [25]:
!pip install scrapy
!pip install crochet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **2.0.1 Scrapping the Webpage**

In [29]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

Exception in thread CrochetReactor:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/crochet/_eventloop.py", line 372, in <lambda>
    target=lambda: self._reactor.run(installSignalHandlers=False),
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 1317, in run
    self.startRunning(installSignalHandlers=installSignalHandlers)
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 1299, in startRunning
    ReactorBase.startRunning(cast(ReactorBase, self))
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 843, in startRunning
    raise error.ReactorNotRestartable()
twisted.internet.error.ReactorNotRestartable



In [30]:
class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from `wikiquote` by 
    Maynard James Keenan and save to json file"""
    name = "MJKQuotesToCsv"
    start_urls = [
        'https://en.wikiquote.org/wiki/Swahili_proverbs',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

### **2.0.2 Initializing the Crawler**

In [32]:
process = CrawlerProcess()
process.crawl(QuotesToCsv)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 11:14:37 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-27 11:14:37 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{}
2022-10-27 11:14:37 [scrapy.crawler] INFO: Overridden settings:
{}
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-27 11:14:37 [scrapy.

ReactorNotRestartable: ignored

## **3.0 Scrapy 3.0**

### **3.0.0 Libraries**

In [33]:
import scrapy
from scrapy.crawler import CrawlerProcess

### **3.0.1 The Spider**

In [34]:
class MarilynMansonQuotes(scrapy.Spider):
    name = "MarilynMansonQuotes"
    start_urls = [
        'https://en.wikiquote.org/wiki/Swahili_proverbs',
    ]
    
    def parse(self, response):
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}

process = CrawlerProcess()
process.crawl(MarilynMansonQuotes)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 11:22:47 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-27 11:22:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{}
2022-10-27 11:22:47 [scrapy.crawler] INFO: Overridden settings:
{}
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-27 11:22:47 [scrapy.

ReactorNotRestartable: ignored

### **3.0.2 Processing pipeline**

In [43]:
import re

class ExtractFirstLine(object):
def process_item(self, item, spider):
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])
        return {'quote': first_line}
  
    def __remove_html_tags__(self, text):
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

IndentationError: ignored

https://www.mikulskibartosz.name/how-to-scrape-a-single-web-page-using-scrapy-in-jupyter-notebook/